In [1]:
import sys

import numpy as np
from imageio import imread, imwrite
from scipy.ndimage import convolve
import imageio.v2 as imageio
import cv2

# tqdm is not strictly necessary, but it gives us a pretty progress bar
# to visualize progress.
from tqdm import trange

In [2]:
#my varaibles
inputAdr = "./SamplesDataset/"
picName = "Baby"
outputAdr = "./Output/"

In [3]:
def calc_gmap(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  gradient_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 3)
  gradient_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = 3)

  gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

  gradient_normalized = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  return gradient_normalized

In [4]:
def calc_mask(img1, img2):
  A = img1 + img2
  B = abs(img1 - img2)

  threshA, _ = cv2.threshold(A, 0, 255, cv2.THRESH_OTSU)
  threshB, _ = cv2.threshold(B, 0, 255, cv2.THRESH_OTSU)

  return (A>threshA) & (B<threshB)

In [5]:
#Energy map
def calc_energy(img, SM, DM):

    GM = calc_gmap(img)
    MaskSD = calc_mask(SM,DM)
    MaskSG = calc_mask(SM,GM)
    MaskSDG = MaskSD * MaskSG
    MaskSDG = MaskSDG.astype(np.uint8)

    SMimproved = MaskSDG | SM

    energy_map = (GM/5)*2 + (DM/5)*2 + (SMimproved/5)

    r, c, _ = img.shape
    for i in range(r):
      for j in range(c):
        if (j% 2==0):
          energy_map[i,j]/=3

    energy_mapInt8 = energy_map.astype(np.uint8)
    imwrite(f"{outputAdr}/{picName}_EMapComplete.png", energy_mapInt8)
    return energy_map

In [6]:
#Finding the seam with least energy
def minimum_seam(img, SM, DM):
    r, c, _ = img.shape
    energy_map = calc_energy(img, SM, DM)

    M = energy_map.copy()
    M = M.astype(np.uint32)
    backtrack = np.zeros_like(M, dtype=int)

    for i in range(1, r):
        for j in range(0, c):
            # Handle the left edge of the image, to ensure we don't index -1
            if j == 0:
                idx = np.argmin(M[i - 1, j:j + 2])
                backtrack[i, j] = idx + j
                min_energy = M[i - 1, idx + j]
            else:
                idx = np.argmin(M[i - 1, j - 1:j + 2])
                backtrack[i, j] = idx + j - 1
                min_energy = M[i - 1, idx + j - 1]

            M[i, j] += min_energy

    return M, backtrack

In [7]:
#Deleting the pixels from the seam with the least energy
def carve_column(img, SM, DM):
    r, c, _ = img.shape

    M, backtrack = minimum_seam(img, SM, DM)

    # Create a (r, c) matrix filled with the value True
    # We'll be removing all pixels from the image which
    # have False later
    mask = np.ones((r, c), dtype=bool)

    # Find the position of the smallest element in the
    # last row of M
    j = np.argmin(M[-1])

    for i in reversed(range(r)):
        # Mark the pixels for deletion
        mask[i, j] = False
        j = backtrack[i, j]

    # Since the image has 3 channels, we convert our
    # mask to 3D
    mask3D = np.stack([mask] * 3, axis=2)

    # Delete all the pixels marked False in the mask,
    # and resize it to the new image dimensions
    img = img[mask3D].reshape((r, c - 1, 3))
    SM = SM[mask].reshape((r, c - 1))
    DM = DM[mask].reshape((r, c - 1))

    return img, SM, DM

In [8]:
#Repeat for every column
def crop_c(img, SM, DM, scale_c):
    r, c, _ = img.shape
    new_c = int(scale_c * c)

    for i in trange(c - new_c): # use range if you don't want to use tqdm
        img, SM, DM = carve_column(img, SM, DM)

    return img

In [11]:
def main():
    scale = 0.5
    in_filename = f"{inputAdr}{picName}/{picName}.png"
    out_filename = f"{outputAdr}/{picName}.png"
    SM = imageio.imread(f"{inputAdr}{picName}/{picName}_SMap.png")

    DM = imageio.imread(f"{inputAdr}{picName}/{picName}_DMap.png")
    _, DM = cv2.threshold(DM, 0, 255, cv2.THRESH_OTSU)

    img = imageio.imread(in_filename)
    out = crop_c(img, SM, DM, scale)
    imwrite(out_filename, out)

if __name__ == '__main__':
    main()

  0%|          | 0/207 [00:00<?, ?it/s]

100%|██████████| 207/207 [01:15<00:00,  2.75it/s]
